In [1]:
# Imports
import torch
import os 
import sys
from dask_jobqueue import SLURMCluster
from distributed import Client
from pathlib import Path

import do_cbohealth_samplingcbo as file

In [2]:
# Args setup
ntrial = 50
n = 100
ntest = 100

In [3]:
# Cluster creation
cluster = SLURMCluster(
    n_workers=0,
    memory="32GB",
    processes=1,
    cores=1,
    scheduler_options={
        "dashboard_address": ":11111",
        "allowed_failures": 10
    },
    job_cpu=1,
    walltime="3:0:0",
    
    job_extra_directives = ["-p medium,fast,cpu"],
)
cluster.adapt(minimum=0, maximum=200)
client = Client(cluster)

2024-09-22 19:49:18,690 - distributed.scheduler - ERROR - Task main-db1871ed3fc8c3ccc06a7ded7fdd3323 marked as failed because 11 workers died while trying to run it
2024-09-22 19:49:34,847 - distributed.scheduler - ERROR - Task main-4b994e2d35484dc7a8256598c3edffb0 marked as failed because 11 workers died while trying to run it
2024-09-22 19:49:46,374 - distributed.scheduler - ERROR - Task main-38447a722dc7db4a1b68fd4737b39288 marked as failed because 11 workers died while trying to run it
2024-09-22 19:49:46,763 - distributed.scheduler - ERROR - Task main-411e99c4b195e194706dac71f94cbf5a marked as failed because 11 workers died while trying to run it


In [4]:
# Submitting jobs
futures = []
for seed in range(ntrial):
        f = client.submit(file.main,
                          seed,
                          n,ntest
                         )
        futures += [f]

In [5]:
# Check on futures
#futures

In [6]:
# Getting results
results = client.gather(futures)

_LinAlgError: linalg.cholesky: The factorization could not be completed because the input is not positive-definite (the leading minor of order 88 is not positive-definite).

In [7]:
# Closing client
client.close()
cluster.close()

In [ ]:
# Saving results
torch.save(obj = results,
           f = "cbohealth_samplingcbo_ntrial={0}_n={1}.pt".format(ntrial,n))